# PART I

-----------------------

# Creating the dataset to play with.

- Merge 4 datasets with different lenght by date
- Google searches dataset:
    - There are 3 columns: Date, keywords, and score, grouped by week. It needs to be structured like a column for each keyword, and rows with the score (the score will be the trend index of Google Trends).
    
- Spanish news (3 datasets):
    - There are 3 columns: Date, keyword and sentiment. The same keyword can appear many times the same day.
    - A column counting occurrences per day of a keyword must be created, also a column with mean sentiment per day.
    - Then a column called score will be created, multiplying occurrences * mean.
    - Then the final datset will be Date, as much columns as keywords, every column will have the score mentioned above as rows. Everything grouped by week.

In [1]:
import pandas as pd

In [2]:
df_political=pd.read_csv("./input/dashboard_spanish_news_political.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_political.sort_values(by=["Date"],inplace=True)

df_economical=pd.read_csv("./input/dashboard_spanish_news_economical.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_economical.sort_values(by=["Date"],inplace=True)

df_social=pd.read_csv("./input/dashboard_spanish_news_social.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_social.sort_values(by=["Date"],inplace=True)

df_google=pd.read_csv("./input/data_pytrends.csv",index_col=[0])
df_google.sort_values(by=["date"],inplace=True)

# Dataset previews:

- With news, I need to count occurrences, calculate the average of sentiment, multiply both to create a new column.


In [3]:
datasets_raw=[df_political, df_social, df_economical, df_google]
for d in datasets_raw:
    
    display(d.tail())
    column_list= d.columns
    print("Unique keywords: ", d[column_list[0]].unique(),"\n","\n",
         "Shape: ", d.shape,"\n","\n"
         )

,political,Date,Sentiment
55731,juicio,2020-12-20,-4.49
55735,juicio,2020-12-20,-1.08
55734,juicio,2020-12-20,-8.04
55732,juicio,2020-12-20,-6.47
55773,ejercito,2020-12-20,-1.78


Unique keywords:  ['juicio' 'ejercito' 'vigilancia' 'protestas' 'inestabilidad_politica'
 'seguridad_nacional' 'corrupcion' 'refugiados' 'rebelion' 'terrorismo'
 'extremismo' 'precio_petroleo'] 
 
 Shape:  (55899, 3) 
 



,social,Date,Sentiment
42788,emergencia_sanitaria,2020-12-20,-4.60
42787,emergencia_sanitaria,2020-12-20,-4.65
42786,emergencia_sanitaria,2020-12-20,-2.00
42797,emergencia_sanitaria,2020-12-20,-3.45
42877,enfermedades_muy_infecciosas,2020-12-20,-0.14


Unique keywords:  ['enfermedades_muy_infecciosas' 'vacunas' 'emergencia_sanitaria'
 'inmigracion' 'agresion_sexual' 'ciencia' 'emprendimiento'
 'precio_vivienda' 'energias_renovables' 'censura_en_medios' 'subsidios'
 'racismo'] 
 
 Shape:  (42878, 3) 
 



,economical,Date,Sentiment
100,stock_market,2020-12-20,-4.16
99,stock_market,2020-12-20,-1.71
98,stock_market,2020-12-20,-2.56
966,incertidumbre_economica,2020-12-20,-5.53
950,incertidumbre_economica,2020-12-20,-2.92


Unique keywords:  ['pobreza' 'incertidumbre_economica' 'stock_market'
 'macroeconomia_deuda_y_vulnerabilidad' 'desempleo'
 'job_quality_&_labor_market_performance' 'libre_comercio'
 'inflacion_economica' 'precio_petroleo' 'quiebra_economica'
 'crecimiento_economico' 'banco_mundial' 'finanzas_y_bancos'
 'prosperidad_economica_y_finanzas'] 
 
 Shape:  (160349, 3) 
 



,keyword,date,trend_index
7139,yoga,2020-12-13,39
5915,examenes,2020-12-13,33
2243,podemos,2020-12-13,18
917,comparecencia,2020-12-13,4
8261,en familia,2020-12-13,62


Unique keywords:  ['zoom' 'yoga' 'crisis economica' 'badi' 'erte' 'vox' 'examenes'
 'refugiados' 'estado de alarma' 'psoe' 'idealista' 'paro' 'menu escolar'
 'meetic' 'uber' 'autonomo' 'bullying' 'hipoteca' 'sepe' 'taxi' 'hangouts'
 'corrupcion' 'tinder' 'en casa' 'clases online' 'piso barato' 'infeccion'
 'videollamada' 'pandemia' 'negocio online' 'crisis politica'
 'productividad' 'independentismo' 'videoconferencia' 'medico'
 'coronavirus' 'emprendimiento' 'cursos online' 'residencia ancianos'
 'skype' 'en familia' 'meditacion' 'barometro' 'remoto' 'desescalada'
 'comparecencia' 'teletrabajo' 'formacion' 'embarazo' 'pp' 'nacionalismo'
 'amazon' 'just eat' 'inmigracion' 'comedor social' 'compartir piso'
 'steam' 'mas pais' 'desempleo' 'uber eats' 'banco alimentos'
 'manifestacion' 'caritas' 'cruz roja' 'hbo' 'hacer deporte' 'pnv'
 'deshaucio' 'ayuda alquiler' 'podemos' 'disney' 'glovo' 'netflix'
 'deliveroo' 'bildu' 'ciudadanos' 'cabify' 'erc' 'divorcio' 'protesta'
 'compromis'] 
 
 

With Google searches, what we have is the Trending index. Score 100 means that from the requested period of time, in this week the maximum of occurrences were reached, and everything else get normalize by that peak. It is not possible to request the real number of occurrences.

# Data Processing

### Working with the Google dataset: 

- Creating a column for each keyword with the trend_index value.

I'm going to create a dataframe with the set of dates, and append the score of keywords to this dataset (kind of get dummies, but I can't do that, I have already the units I want for each column)

In [4]:
# creating df
df_google_dates=pd.DataFrame()

# creating the Date column in new dataset
df_google_dates["date"]=list(set(df_google["date"]))
df_google_dates["date"]=pd.to_datetime(df_google_dates["date"])
df_google_dates.sort_values(by=["date"],inplace=True)
df_google_dates.tail()

,date
98,2020-11-15
0,2020-11-22
83,2020-11-29
44,2020-12-06
73,2020-12-13


In [5]:
# dates are a pain in the ass, you know right?
df_google_dates.date.dtypes

dtype('<M8[ns]')

 - append to the empty dataframe with dates

In [6]:
# Creating the new columns. Trend index with the name of the corresponding keyword
keyword_list=list(set(df_google["keyword"]))
keyword_list.sort()
for k in keyword_list:
    df_google_dates[k]=df_google[(df_google['keyword'] == k)]["trend_index"].tolist()
#df_google_dates.index=df_google_dates["date"]
#df_google_dates.drop(columns="date",inplace=True)
df_google_dates.tail()

,date,amazon,autonomo,ayuda alquiler,badi,banco alimentos,barometro,bildu,bullying,cabify,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
98,2020-11-15,81,49,27,26,100,47,47,31,7,...,23,16,82,46,76,10,7,8,42,23
0,2020-11-22,98,42,21,39,37,35,22,20,8,...,23,12,66,49,89,10,7,7,42,20
83,2020-11-29,85,54,22,17,14,25,18,22,11,...,34,15,76,37,81,7,7,7,34,18
44,2020-12-06,90,31,21,41,10,33,10,23,10,...,28,12,98,48,82,10,8,7,39,16
73,2020-12-13,88,44,25,31,14,13,6,18,13,...,28,10,79,35,57,9,9,6,39,18


**This will be our target in the ML regression"**: Unemployment

In [7]:
df_google_dates.rename(columns={'desempleo':"unemployment"}, inplace=True)

### Working with news datasets. 

I'll need to:

- Create a column for each keyword.
- Count occurrences of that keyword.
- Measure average sentiment.
- Group data by week, starting on sunday, to merge with the Google dataset.
- Combine occurrences and sentiment into one column representative of both, for each keyword.

In [8]:
# let's pplay with the 1st dataset and a random keyword, for instance
df_political[df_political["political"]=="juicio"].head()

,political,Date,Sentiment
15784,juicio,2019-01-01,-6.08
15773,juicio,2019-01-01,0.18
15774,juicio,2019-01-01,0.18
15775,juicio,2019-01-01,-1.20
15776,juicio,2019-01-01,-1.20


- So, I need to measure the average of sentiment of each keyword per day

In [9]:
df_political[df_political["political"]=="juicio"].groupby("Date").mean().head()

,Sentiment
Date,
2019-01-01,-2.930000
2019-01-02,-3.986667
2019-01-03,-2.350000
2019-01-04,-4.822500
2019-01-05,-4.723333


- Also, counting occurrences of that keyword

In [10]:
df_political[df_political["political"]=="juicio"].groupby("Date").count().head()

,political,Sentiment
Date,,
2019-01-01,14,14
2019-01-02,18,18
2019-01-03,40,40
2019-01-04,16,16
2019-01-05,60,60


- Let's use an aggregate to perform both

In [11]:
df2=df_political[df_political["political"]=="juicio"].groupby(["Date"]).agg(['count','mean'])
# erase multiindex
df2.columns=df2.columns.droplevel(0)
df2.tail()

,count,mean
Date,,
2020-12-16,21,-5.710952
2020-12-17,13,-4.783077
2020-12-18,12,-8.670833
2020-12-19,8,-5.951250
2020-12-20,11,-3.266364


- Great, now let's resample by week, starting on Sunday, like the Google Searches dataset

In [12]:
df2.index = pd.to_datetime(df2.index)
df2 = df2.resample('W-SUN').mean() #weekly totals
# score is how we are going to measure the keywords
df2["score"]=df2["count"]*df2["mean"]
df2.tail()

,count,mean,score
Date,,,
2020-11-22,9.000000,-4.722266,-42.500395
2020-11-29,17.000000,-5.013151,-85.223565
2020-12-06,20.285714,-4.461923,-90.513293
2020-12-13,9.428571,-5.743085,-54.149087
2020-12-20,14.857143,-5.962872,-88.591243


ok, now I know how to do it, then let's continue creating a function to perform this for every keyword in every Spanish news datasets.

- I need to create an empty dataframe.
- loop for each keyword from a set of keywords.
- perform what i did before for all keywords.
- concat to the mepty dataframe.
- put all this in a function.

In [13]:
# pending of erasing this and writing it in a separate script
def creating_dataset(df,column):
    '''
    Column is the column in which are allocated the keywords, for every case: political, social and economical columns
    '''
    df["Date"]=pd.to_datetime(df["Date"])
    # list of new columns
    list_keywords=list(set(df[column]))
    # creating empty dataframe to append info
    df_final=pd.DataFrame()
    df_final["date"]=list(set(df["Date"]))
    
    for k in list_keywords:
        # creating a new dataframe for every keyword in the column, getting the occurrences of keyword and mean of sentiment
        df4=pd.DataFrame()
        df4=df[df[column]==k].groupby(["Date"]).agg(['count','mean'])
        # erase multiindex
        df4.columns=df4.columns.droplevel(0)
        # this will be our score, occurrences * mean 
        df4[k]=df4["count"]*df4["mean"]
        # date column to perform the join by it
        df4["date"]=df4.index
        df4.drop(columns=["count","mean"],inplace=True)
        # this is where we combine the empty dataset, every keyword in its place
        df_final=df_final.merge(df4,how='left', left_on='date', right_on='date')

    # resampling 
    # this is weird: transform date column in index, group by, then transform again index in column, to make the further join
    df_final.index=df_final["date"]
    df_final = df_final.resample('W-SUN').mean() #weekly totals
    df_final.sort_values(by="date", ascending=True, inplace=True)
    df_final["date"]=df_final.index
    df_final.reset_index(drop=True, inplace=True)
    
    
    return df_final

In [14]:
dfp = creating_dataset(df_political,"political")
dfs = creating_dataset(df_social,"social")
dfe = creating_dataset(df_economical,"economical")

In [15]:
dfe.date.dtypes

dtype('<M8[ns]')

In [16]:
dfe.shape

(103, 15)

In [17]:
dfe.tail()

,prosperidad_economica_y_finanzas,banco_mundial,incertidumbre_economica,finanzas_y_bancos,inflacion_economica,quiebra_economica,desempleo,crecimiento_economico,job_quality_&_labor_market_performance,macroeconomia_deuda_y_vulnerabilidad,libre_comercio,precio_petroleo,stock_market,pobreza,date
98,NaN,NaN,-254.740000,0.42,NaN,-2.870,-1.998333,-2.690000,-13.128000,-4.920000,NaN,NaN,-8.515714,-17.824286,2020-11-22
99,-6.47,NaN,-294.670000,NaN,NaN,NaN,-2.448000,-0.760000,-6.680000,-10.077500,-3.25,NaN,-19.915714,-22.805714,2020-11-29
100,-4.47,NaN,-354.131429,NaN,-6.310,-1.010,-5.247500,-3.766667,-1.948000,-13.760000,NaN,NaN,-6.292857,-14.445714,2020-12-06
101,NaN,-2.87,-337.195714,NaN,-1.875,NaN,-3.762500,-2.610000,-3.913333,-0.543333,-66.81,-2.64,-7.792857,-28.698571,2020-12-13
102,NaN,NaN,-286.014286,NaN,3.210,-2.045,-6.856667,NaN,-0.836667,-6.055000,-6.18,NaN,-9.238571,-29.917143,2020-12-20


In [18]:
dfs.shape

(103, 13)

In [19]:
dfs.tail()

,precio_vivienda,ciencia,inmigracion,racismo,subsidios,enfermedades_muy_infecciosas,vacunas,censura_en_medios,emprendimiento,emergencia_sanitaria,energias_renovables,agresion_sexual,date
98,-4.32,-1.501667,NaN,NaN,NaN,-31.867143,NaN,NaN,-1.555,-91.004286,NaN,NaN,2020-11-22
99,0.36,-8.197143,1.840000,NaN,NaN,-41.211429,-1.00,NaN,8.960,-111.404286,NaN,-7.465,2020-11-29
100,NaN,-15.538333,4.112500,NaN,NaN,-32.165714,2.85,NaN,-4.700,-157.940000,-3.66,-13.790,2020-12-06
101,NaN,-7.207500,-7.586667,NaN,NaN,-44.785714,-3.13,NaN,-0.810,-123.848571,0.62,-8.305,2020-12-13
102,NaN,-3.338333,-2.624000,NaN,NaN,-25.485714,-1.00,NaN,NaN,-106.138571,NaN,NaN,2020-12-20


In [20]:
dfs.shape

(103, 13)

In [21]:
dfe.tail()

,prosperidad_economica_y_finanzas,banco_mundial,incertidumbre_economica,finanzas_y_bancos,inflacion_economica,quiebra_economica,desempleo,crecimiento_economico,job_quality_&_labor_market_performance,macroeconomia_deuda_y_vulnerabilidad,libre_comercio,precio_petroleo,stock_market,pobreza,date
98,NaN,NaN,-254.740000,0.42,NaN,-2.870,-1.998333,-2.690000,-13.128000,-4.920000,NaN,NaN,-8.515714,-17.824286,2020-11-22
99,-6.47,NaN,-294.670000,NaN,NaN,NaN,-2.448000,-0.760000,-6.680000,-10.077500,-3.25,NaN,-19.915714,-22.805714,2020-11-29
100,-4.47,NaN,-354.131429,NaN,-6.310,-1.010,-5.247500,-3.766667,-1.948000,-13.760000,NaN,NaN,-6.292857,-14.445714,2020-12-06
101,NaN,-2.87,-337.195714,NaN,-1.875,NaN,-3.762500,-2.610000,-3.913333,-0.543333,-66.81,-2.64,-7.792857,-28.698571,2020-12-13
102,NaN,NaN,-286.014286,NaN,3.210,-2.045,-6.856667,NaN,-0.836667,-6.055000,-6.18,NaN,-9.238571,-29.917143,2020-12-20


-----------------------------------------
-----------------------------------------

In [22]:
datasets = [ dfp, dfe, dfs,df_google_dates]

c=0
for d in datasets:
    c+=len(d.columns)

print("Number of attributes I have to play with: ",c)

#print(len(datasets))

Number of attributes I have to play with:  123


----------------------------------------------
----------------------------------------------

-  **don't worry by NAN, i'll deal with that later**

### Merging all datasets

- Create an empty dataframe.
- Create a column for it all dates
- Use a left join using the date column, to append in the proper place the keywords of all the other datasets

In [23]:
from datetime import datetime, date

In [24]:
# first of all let's discover the later date we have
#datasets = [ dfp, dfe, dfs,df_google_dates] 
date2 = []
for d in datasets:
    date2. append(d.date.max()) # get the later date of every dataset
    
date2 = max(date2) # get the higher of the latest
date2

Timestamp('2020-12-20 00:00:00')

In [25]:
# creating final dataset with everything
date1 = '2019-01-01'
 
mydates = pd.date_range(date1, date2, freq="W").tolist()
df_final=pd.DataFrame()
df_final["date"]=mydates
df_final['date']=pd.to_datetime(df_final["date"])
df_final.tail()

,date
98,2020-11-22
99,2020-11-29
100,2020-12-06
101,2020-12-13
102,2020-12-20


In [26]:
df_final.date.dtypes

dtype('<M8[ns]')

In [27]:
# datasets = [ dfp, dfe, dfs,df_google_dates]  already referenced before 

In [28]:
# Merging!
for d in datasets:
    df_final=df_final.merge(d,how='left', left_on="date", right_on="date",suffixes=["_1","_2"])

In [29]:
df_final.shape

(103, 120)

In [30]:
df_final.columns # basically all reference one by one before

Index(['date', 'seguridad_nacional', 'vigilancia', 'corrupcion_1', 'ejercito',
       'rebelion', 'extremismo', 'refugiados_1', 'terrorismo',
       'precio_petroleo_1',
       ...
       'taxi', 'teletrabajo', 'tinder', 'uber', 'uber eats',
       'videoconferencia', 'videollamada', 'vox', 'yoga', 'zoom'],
      dtype='object', length=120)

In [31]:
df_final=df_final.fillna(0)

In [32]:
df_final.to_csv("./input/dataset_final.csv")

-------------------------------

# Part IIb
### Processing dataset before applying ML.

It's time to process our last df before applying ML. Process for what? Well, I'm going to move my target column 4 weeks ahead, so the rows of the dataset will be related with the rows of my target, 4 weeks in advance.

Thanks to that, the last row of my dataset (today 2020.12.20) will infer my target column the day 2021.01.17.

In [33]:
import pandas as pd
from datetime import timedelta

In [34]:
dataset=pd.read_csv("input/dataset_final.csv",index_col=[0])

- List of features I have to play with:

In [35]:
dataset.tail()

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
98,2020-11-22,-1.505714,-4.705000,-25.255714,-15.242000,-5.103333,0.00,-4.363333,-4.060000,0.00,...,23.0,12.0,66.0,49.0,89.0,10.0,7.0,7.0,42.0,20.0
99,2020-11-29,-6.328000,-9.928571,-43.687143,-20.334286,-2.236667,0.00,-4.660000,-15.684000,0.00,...,34.0,15.0,76.0,37.0,81.0,7.0,7.0,7.0,34.0,18.0
100,2020-12-06,-5.020000,-7.828333,-60.852857,-12.138571,-22.630000,0.00,-6.390000,-12.983333,0.00,...,28.0,12.0,98.0,48.0,82.0,10.0,8.0,7.0,39.0,16.0
101,2020-12-13,-2.935000,-6.966667,-19.585714,-11.297143,-2.640000,-7.67,-3.157500,-6.036667,-2.64,...,28.0,10.0,79.0,35.0,57.0,9.0,9.0,6.0,39.0,18.0
102,2020-12-20,-1.913333,-4.851667,-46.031429,-23.021429,-1.535000,2.12,-5.895000,-9.918000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Sliding time

- I'm going to create a new dataset with *unemployment*, my target column and date
- Remove *unemployment* from the old dataset
- In the old dataset, add 4 weeks to the date rows. In this way, everythin happening right now will be linked with unemployment 4 weeks ahead
- Merge the *unemployment* and date dataset with the old one with changed dates

Summing up, change the dates 4 weeks ahead of everything but *unemployment* column

In [36]:
dataset["date"]=pd.to_datetime(dataset["date"])
df_unemployment=pd.DataFrame(dataset[["date", "unemployment"]])
dataset.drop(columns="unemployment", inplace=True)

In [37]:
dataset["date"]=dataset["date"].apply(lambda x: x+timedelta(weeks=4))
dataset.tail()

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
98,2020-12-20,-1.505714,-4.705000,-25.255714,-15.242000,-5.103333,0.00,-4.363333,-4.060000,0.00,...,23.0,12.0,66.0,49.0,89.0,10.0,7.0,7.0,42.0,20.0
99,2020-12-27,-6.328000,-9.928571,-43.687143,-20.334286,-2.236667,0.00,-4.660000,-15.684000,0.00,...,34.0,15.0,76.0,37.0,81.0,7.0,7.0,7.0,34.0,18.0
100,2021-01-03,-5.020000,-7.828333,-60.852857,-12.138571,-22.630000,0.00,-6.390000,-12.983333,0.00,...,28.0,12.0,98.0,48.0,82.0,10.0,8.0,7.0,39.0,16.0
101,2021-01-10,-2.935000,-6.966667,-19.585714,-11.297143,-2.640000,-7.67,-3.157500,-6.036667,-2.64,...,28.0,10.0,79.0,35.0,57.0,9.0,9.0,6.0,39.0,18.0
102,2021-01-17,-1.913333,-4.851667,-46.031429,-23.021429,-1.535000,2.12,-5.895000,-9.918000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# now let's merge again. outter to get all from both datasets
df=pd.merge(dataset,df_unemployment,how='outer', on=["date"],suffixes=(None,None))
df.sort_values(by=["date"],inplace=True)
df=df.fillna(0)

# great, everything but "unemployment" has been moved 4 weeks ahead
df.head() # go to the right to appreciate it

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
103,2019-01-06,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
104,2019-01-13,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
105,2019-01-20,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
106,2019-01-27,0.000,0.000,0.000,0.000000,0.000000,0.000000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
0,2019-02-03,-30.555,-31.525,-92.164,-20.473333,-21.453333,-29.993333,-6.425,-82.14,0.0,...,1.0,57.0,27.0,12.0,2.0,2.0,34.0,47.0,5.0,18.0


In [39]:
df.tail() # go to the right to appreciate it

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
98,2020-12-20,-1.505714,-4.705000,-25.255714,-15.242000,-5.103333,0.00,-4.363333,-4.060000,0.00,...,12.0,66.0,49.0,89.0,10.0,7.0,7.0,42.0,20.0,0.0
99,2020-12-27,-6.328000,-9.928571,-43.687143,-20.334286,-2.236667,0.00,-4.660000,-15.684000,0.00,...,15.0,76.0,37.0,81.0,7.0,7.0,7.0,34.0,18.0,0.0
100,2021-01-03,-5.020000,-7.828333,-60.852857,-12.138571,-22.630000,0.00,-6.390000,-12.983333,0.00,...,12.0,98.0,48.0,82.0,10.0,8.0,7.0,39.0,16.0,0.0
101,2021-01-10,-2.935000,-6.966667,-19.585714,-11.297143,-2.640000,-7.67,-3.157500,-6.036667,-2.64,...,10.0,79.0,35.0,57.0,9.0,9.0,6.0,39.0,18.0,0.0
102,2021-01-17,-1.913333,-4.851667,-46.031429,-23.021429,-1.535000,2.12,-5.895000,-9.918000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
df.reset_index(drop=True,inplace=True)
# lets remove the first 4 rows. We don't have data in our X, just in our target, so it's redundant
df.drop([0,1,2,3], inplace=True)

In [41]:
df.to_csv("input/dataset_final_processed.csv")
df

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
4,2019-02-03,-30.555000,-31.525000,-92.164000,-20.473333,-21.453333,-29.993333,-6.425000,-82.140000,0.00,...,1.0,57.0,27.0,12.0,2.0,2.0,34.0,47.0,5.0,18.0
5,2019-02-10,-5.313333,-67.150000,-168.828571,-31.360000,-17.413333,-2.260000,-15.266667,-11.586667,0.00,...,1.0,45.0,39.0,9.0,4.0,1.0,27.0,44.0,4.0,22.0
6,2019-02-17,-3.220000,-49.556000,-106.468571,-23.700000,-26.305000,-5.644000,0.000000,-145.260000,3.72,...,0.0,44.0,100.0,14.0,3.0,2.0,15.0,46.0,3.0,20.0
7,2019-02-24,-16.080000,-14.530000,-106.342857,-25.508571,-26.570000,-4.226667,1.276000,-34.084000,0.00,...,1.0,45.0,83.0,10.0,3.0,2.0,11.0,50.0,4.0,21.0
8,2019-03-03,-1.480000,-21.495000,-112.497143,-6.876667,-28.106667,0.000000,-20.310000,-24.512000,0.00,...,3.0,56.0,37.0,10.0,3.0,2.0,13.0,51.0,3.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,2020-12-20,-1.505714,-4.705000,-25.255714,-15.242000,-5.103333,0.000000,-4.363333,-4.060000,0.00,...,12.0,66.0,49.0,89.0,10.0,7.0,7.0,42.0,20.0,0.0
103,2020-12-27,-6.328000,-9.928571,-43.687143,-20.334286,-2.236667,0.000000,-4.660000,-15.684000,0.00,...,15.0,76.0,37.0,81.0,7.0,7.0,7.0,34.0,18.0,0.0
104,2021-01-03,-5.020000,-7.828333,-60.852857,-12.138571,-22.630000,0.000000,-6.390000,-12.983333,0.00,...,12.0,98.0,48.0,82.0,10.0,8.0,7.0,39.0,16.0,0.0
105,2021-01-10,-2.935000,-6.966667,-19.585714,-11.297143,-2.640000,-7.670000,-3.157500,-6.036667,-2.64,...,10.0,79.0,35.0,57.0,9.0,9.0,6.0,39.0,18.0,0.0


----------------------------------

# PART III
# ML regression


![alt](https://i.pinimg.com/originals/8c/5c/59/8c5c5923c547867c1615402754d9eb27.gif)
-----------------------------------

*This jupyter is the explanation. The *real thing* is available in the **Cloud Function** folder. 


----------------------------------

I'm going to:

- Normalise the dataset before applying ML. 
- Use **RFE** (Recursive Feature Elimination) to select the best 15 columns to work with, from the 133 I have.
- Use the fastest regression I can. Speed over performance due to Cloud Function requirements.
- Perform cross validation. Unfortunately I have no enough data for train/validation/test
- Once my model is trained, infer data in the whole dataset, to have a dashboard comparing *real keyword* vs *infered keyword*.

Let's have fun :D

In [42]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA as pca
from scipy import stats
from sklearn.covariance import EllipticEnvelope
import matplotlib.pyplot as plt
import matplotlib.font_manager

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

In [43]:
df=pd.read_csv("input/dataset_final_processed.csv",index_col=[0])

In [44]:
print("I have ", len(df.columns)-1, " columns to play with")#columns=list(df.columns) 

I have  119  columns to play with


In [45]:
df.reset_index(drop=True, inplace=True) # very important
df

,date,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,precio_petroleo_1,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
0,2019-02-03,-30.555000,-31.525000,-92.164000,-20.473333,-21.453333,-29.993333,-6.425000,-82.140000,0.00,...,1.0,57.0,27.0,12.0,2.0,2.0,34.0,47.0,5.0,18.0
1,2019-02-10,-5.313333,-67.150000,-168.828571,-31.360000,-17.413333,-2.260000,-15.266667,-11.586667,0.00,...,1.0,45.0,39.0,9.0,4.0,1.0,27.0,44.0,4.0,22.0
2,2019-02-17,-3.220000,-49.556000,-106.468571,-23.700000,-26.305000,-5.644000,0.000000,-145.260000,3.72,...,0.0,44.0,100.0,14.0,3.0,2.0,15.0,46.0,3.0,20.0
3,2019-02-24,-16.080000,-14.530000,-106.342857,-25.508571,-26.570000,-4.226667,1.276000,-34.084000,0.00,...,1.0,45.0,83.0,10.0,3.0,2.0,11.0,50.0,4.0,21.0
4,2019-03-03,-1.480000,-21.495000,-112.497143,-6.876667,-28.106667,0.000000,-20.310000,-24.512000,0.00,...,3.0,56.0,37.0,10.0,3.0,2.0,13.0,51.0,3.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2020-12-20,-1.505714,-4.705000,-25.255714,-15.242000,-5.103333,0.000000,-4.363333,-4.060000,0.00,...,12.0,66.0,49.0,89.0,10.0,7.0,7.0,42.0,20.0,0.0
99,2020-12-27,-6.328000,-9.928571,-43.687143,-20.334286,-2.236667,0.000000,-4.660000,-15.684000,0.00,...,15.0,76.0,37.0,81.0,7.0,7.0,7.0,34.0,18.0,0.0
100,2021-01-03,-5.020000,-7.828333,-60.852857,-12.138571,-22.630000,0.000000,-6.390000,-12.983333,0.00,...,12.0,98.0,48.0,82.0,10.0,8.0,7.0,39.0,16.0,0.0
101,2021-01-10,-2.935000,-6.966667,-19.585714,-11.297143,-2.640000,-7.670000,-3.157500,-6.036667,-2.64,...,10.0,79.0,35.0,57.0,9.0,9.0,6.0,39.0,18.0,0.0


# 27.12 Elliptic envelope and dimensionality reduction

- Reduce to 2D and plot graph of outliers
- https://medium.com/analytics-vidhya/a-complete-guide-on-dimensionality-reduction-62d9698013d2

How can i get back the outliers out of the decicions boundary, after a dimensionality reduction?

In [46]:
def outliers_graph(df, outlier_method, outliers_begin, threshold, xmin, xmax):
    
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 100), np.linspace(xmin, xmax, 100))
    Z = outlier_method.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(20, 14))
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, xmax), cmap=plt.cm.Blues_r)
    a = plt.contour(xx, yy, Z, levels=[threshold], linewidths=2, colors='red')
    plt.contourf(xx, yy, Z, levels=[threshold, Z.max()], colors='orange')
    b = plt.scatter(df.iloc[:outliers_begin, 0], df.iloc[:outliers_begin, 1], c='white', s=20, edgecolor='k')
    c = plt.scatter(df.iloc[outliers_begin:, 0], df.iloc[outliers_begin:, 1], c='black', s=20, edgecolor='k')
    plt.axis('tight')
    '''
    # In case I'll use the axis unemployment and 1D proyection
    plt.ylim((-5,105)) # forcing the graph to fit my target range
    plt.title('Elliptic envelope over my target and a 1D proyection of my training set', fontsize=20)
    plt.xlabel('1D proyection', fontsize=16)
    plt.ylabel('Unemployment', fontsize=16)
    #
    '''
    plt.legend(
        [a.collections[0], b, c],
        ['Decision boundary', 'Valid instances', 'Outliers'],
        prop=matplotlib.font_manager.FontProperties(size=34),
        loc='lower right')
    plt.savefig('input/outliers.png')
    plt.show()    

In [47]:
outliers_dect= df.drop(columns=["date","unemployment"])

In [48]:

# this is for having a 2D representation with everything. The result is visually cool
from sklearn import decomposition
pca=decomposition.PCA()
pca.n_components=2
pca_data=pca.fit_transform(outliers_dect)
pca_data=pd.DataFrame(pca_data)
pca_data.rename(columns={0:"a",1:"b"}, inplace=True)
pca_data.shape

(103, 2)

In [49]:
'''
# I think the other method is more logical... I'm quite confused, need to check things out
# This is for having a 2D representation with my target (unemployment), and a 1D proyection of everything else
from sklearn import decomposition
pca=decomposition.PCA()
pca.n_components=1
pca_data=pca.fit_transform(outliers_dect)
pca_data=pd.DataFrame(pca_data)
#pca_data.rename(columns={0:"proyection"})
#pca_data.shape
pca_data.rename(columns={0:"proyections"}, inplace=True)
# So I'll create a 2D representation: of my target, and a 1D proyection of everything else
pca_data["unemployment"]=df["unemployment"]
'''

'\n# I think the other method is more logical... I\'m quite confused, need to check things out\n# This is for having a 2D representation with my target (unemployment), and a 1D proyection of everything else\nfrom sklearn import decomposition\npca=decomposition.PCA()\npca.n_components=1\npca_data=pca.fit_transform(outliers_dect)\npca_data=pd.DataFrame(pca_data)\n#pca_data.rename(columns={0:"proyection"})\n#pca_data.shape\npca_data.rename(columns={0:"proyections"}, inplace=True)\n# So I\'ll create a 2D representation: of my target, and a 1D proyection of everything else\npca_data["unemployment"]=df["unemployment"]\n'

In [50]:
pca_data.head()

,a,b
0,83.773519,1.447543
1,233.044393,-295.299048
2,244.214963,-120.878780
3,216.877194,-114.151077
4,145.190770,-72.912746


In [51]:
# Limpieza de datos: detección de outliers.
outlier_method = EllipticEnvelope().fit(pca_data)
scores_pred = outlier_method.decision_function(pca_data)
threshold = stats.scoreatpercentile(scores_pred, 15) # remove just the heaviest outliers

In [52]:
pca_data.min()

a   -584.697860
b   -295.299048
dtype: float64

In [53]:
df["outliers_score"]=scores_pred
df =df[df["outliers_score"]>=threshold]
df.shape

(87, 121)

In [54]:
# Ratio de outliers.
print("%.3f %%" % (100*len(scores_pred[scores_pred < threshold])/len(scores_pred)), "of the instances were removed")

15.534 % of the instances were removed


In [55]:
# Dibujar gráfica de outliers.
plot_min=int(min(list(pca_data.min())))
plot_max=int(max(list(pca_data.max())))
outliers_graph(pca_data, outlier_method, 150, threshold, plot_min, plot_max)

### First attributes selection: Erasing low variance elements

Let's remove the attibutes (columns) which values are quite constant in the whole dataset, so they're not offering much information when everything else is changing.

In [56]:
from sklearn.feature_selection import VarianceThreshold
def variance_threshold_selector(data, threshold):
    '''
    It removes any attribute (column) than vary less than the percentaje of the threshold
    '''
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [57]:
low_v = df.drop(columns=["date","unemployment","outliers_score" ])

In [58]:
X_raw= variance_threshold_selector(low_v, 5)
X_raw.shape

(87, 109)

In [59]:
low_v.shape

(87, 118)

In [60]:
for c in low_v.columns:
    if c not in X_raw.columns:
        print(c)

precio_petroleo_1
prosperidad_economica_y_finanzas
banco_mundial
inflacion_economica
precio_petroleo_2
subsidios
censura_en_medios
desescalada
hacer deporte


### Normalization / Standarization

Standardizing the features around the center and 0 with a standard deviation of 1 is important when we compare measurements that have different units. Variables that are measured at different scales do not contribute equally to the analysis and might end up creating a bais.

In other hand, Normalization is useful when your data has varying scales and the algorithm you are using does not make assumptions about the distribution of your data

In [61]:
# Normalizacion with sklearn
# remove dates and target
#X_raw = df.drop(columns=["date","unemployment","outliers_score" ]) #already removed in low_v

normalizer = preprocessing.MinMaxScaler()
X = pd.DataFrame(normalizer.fit_transform(X_raw))

#standardizer = preprocessing.StandardScaler()
#X = pd.DataFrame(standardizer.fit_transform(X_raw))

X.columns= X_raw.columns
X.shape
target=df["unemployment"]
X.head(2)

,seguridad_nacional,vigilancia,corrupcion_1,ejercito,rebelion,extremismo,refugiados_1,terrorismo,juicio,inestabilidad_politica,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
0,0.420146,0.640186,0.705565,0.893647,0.579915,0.664755,0.780032,0.438318,0.82097,0.894425,...,0.43,0.01,0.57,0.27,0.12,0.052632,0.166667,0.34,0.870370,0.142857
1,0.765143,0.231892,0.398453,0.823433,0.654679,0.929841,0.701877,0.928255,0.00000,0.778049,...,0.50,0.01,0.45,0.39,0.09,0.105263,0.083333,0.27,0.814815,0.114286


### Validation (nested validation)

I'll validate the algorithoms with the training set. To tune it properly I'll create a substet test from the training set. 

In [62]:
import numpy as np
from math import sqrt
from pprint import pprint
from sklearn import datasets
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

In [63]:
# splitting
X_train=X.iloc[:-4]
target_train=target.iloc[:-4]

# last 4 rows, which are the ones I have no data and I want to infer
X_test=X.iloc[-4:]
target_test=target.iloc[-4:]

In [64]:
# alpha=1 is like a regular regression. for getter performance use elastic net instead (l1&l2 mix).
#regression = Lasso(alpha=0.1)
regression = LinearRegression() # oher model

In [65]:
'''# Métricas de evaluación.
metricas = {
  'MAE': 'neg_mean_absolute_error',
  'RMSE': make_scorer(
          lambda y, y_pred:
            sqrt(mean_squared_error(y, y_pred)),
            greater_is_better=False),
  'MAPE': make_scorer(
          lambda y, y_pred:
            np.mean(np.abs((y - y_pred) / y)) * 100,
            greater_is_better=False)}
# Validación y evaluación del modelo.
evaluacion = cross_validate(regression, X_train, target_train,
                cv = KFold(n_splits=10), scoring = metricas)

# Presentación de los resultados de la evaluación.
#pprint(evaluacion)
for e in evaluacion:
    print(e, evaluacion.get(e).mean(), evaluacion.get(e).std())'''

"# Métricas de evaluación.\nmetricas = {\n  'MAE': 'neg_mean_absolute_error',\n  'RMSE': make_scorer(\n          lambda y, y_pred:\n            sqrt(mean_squared_error(y, y_pred)),\n            greater_is_better=False),\n  'MAPE': make_scorer(\n          lambda y, y_pred:\n            np.mean(np.abs((y - y_pred) / y)) * 100,\n            greater_is_better=False)}\n# Validación y evaluación del modelo.\nevaluacion = cross_validate(regression, X_train, target_train,\n                cv = KFold(n_splits=10), scoring = metricas)\n\n# Presentación de los resultados de la evaluación.\n#pprint(evaluacion)\nfor e in evaluacion:\n    print(e, evaluacion.get(e).mean(), evaluacion.get(e).std())"

In [66]:
## For evaluating multiple scores, use sklearn.model_selection.cross_validate instead.
#rfecv = RFECV(estimator=regression, step=1, min_features_to_select=20, cv=5,scoring=metricas)
#rfecv.fit(X_train, target_train)

In [76]:
# define dataset
# splitting
X_train=X.iloc[:-4]
target_train=target.iloc[:-4]

# last 4 rows, which are the ones I have no data and I want to infer
X_test=X.iloc[-4:]
target_test=target.iloc[-4:]

- MAE => the average error of a measure

Google Trends works oddly. You request a range of time, and it processes the info before delivering. Score 100 means it was the maximum peak in all times, and all the column is normalized by that peak, but we can't see the exactly number of searches.

# Splitting and k-folds validation.

There are not a lot of data, so splitting and shuffling my dataset ends with terrible metrics for a fast linear regression. 

In [69]:
# all but last 4 rows
X_train=X.iloc[:-4]
target_train=target.iloc[:-4]

# last 4 rows, which are the ones I have no data and I want to infer
X_test=X.iloc[-4:]
target_test=target.iloc[-4:]

# Model selection

Here is where I run different models and perform tunning of hyperparameter for better fit, and:

- Divide between train/validation/test
- Train the models with train
- Validate metrics with validation
- Perform inferences with test
- Select the best model of all and train it with the whole dataset

**That's not going to happen... Yet**. This is a leisure project and my main concern is having something that works within a Cloud Function, in my GCP pipelines. It means my baby needs to run with 4GB of RAM as maximum and 60 seconds of timeout, so I will start with the simplest and fastest model I can.

When I get that, myself from the future will refactorize code and improve metrics. 

So, right now, the selected model will be a Lasso regression after a recursive feature elimination. 

- The model will select the 15 best columns to work with.
- After it, Lasso will remove from the equation the less relevants.

It is like a double filter of simplicity that I hope it works

# Selecting best features using RFE

(Recursive Feature Elimination)

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE

https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py

In [94]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [100]:
# My number of features will be a tenth of the available fows
number_features =  1+df.shape[0]//10
estimator = [SVR(kernel="linear") , LinearRegression(),Lasso(alpha=0.1)]
for model in estimator:
    selector = RFE(model, n_features_to_select=number_features, step=1)
    selector = selector.fit(X_train, target_train)
    print(selector.score(X_train, target_train))

0.11100733710626265
0.743059355068528
0.6750546661849839


In [112]:
from sklearn.model_selection import StratifiedKFold

In [115]:
# LO QUE TENÍA ANTES

# alpha=1 is like a regular regression. for getter performance use elastic net instead (l1&l2 mix).
#regression = Lasso(alpha=0.1)
regression = LinearRegression() # oher model
min_features_to_select=1
# neg mean squared error. it always is negative but what you get is the positive representation
rfecv = RFECV(estimator=regression, step=1, min_features_to_select=min_features_to_select, cv=StratifiedKFold(2),scoring='neg_mean_squared_error')
rfecv.fit(X_train, target_train)

# it's unfortunate not having X and target validation, but k-folds cross validation was implemented for that precisely 
rfecv.score(X_train, target_train)

/home/vargas/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)


0.988645277698316

In [122]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
#plt.ylim([0, 1])
plt.show()

# https://machinelearningmastery.com/rfe-feature-selection-in-python/

In [ ]:
# Inferences 4 weeks ahead
rfecv.predict(X_test) 

In [ ]:
#rfecv.predict(X) # this is just for visualization

In [ ]:
len(rfecv.predict(X))
len(df)

In [ ]:
df["inferred_results"]=list(rfecv.predict(X))

# CSV with infered keyword vs real keyword

- Last 4 rows are the truly interesting ones... Foreseeing with 4 weeks in advance.

In [ ]:
result=pd.DataFrame()
result[["date", "real_searches"]]= df[["date","unemployment"]]
result["inferred_results"] = rfecv.predict(X)
result["inferred_results"]=result["inferred_results"].apply(lambda x: 0 if x<0 else round(x,2))
result["inferred_results"]=result["inferred_results"].apply(lambda x: 100 if x>100 else round(x,2))

if result["inferred_results"].min()==0:
    result.drop(result.tail(1).index,inplace=True) # drop last row (better visualization)
result.tail(10)


In [ ]:
# visualization
result.plot(x="date",
            y=["real_searches","inferred_results"],
            figsize=(20,10), 
            kind="bar",
            title="Inferences of Tending Index in Google Trends, with results 4 weeks ahead. This data will be implemented in an automated pipeline for dashboarding in Data Studio",
            grid=True)

## CSV to append weekly the score

In [ ]:
score = pd.DataFrame({"date": [max(df["date"])], 'score': [round(rfecv.score(X_train, target_train),4)]})
score.to_csv("input/weekly_score.csv")
score

## CSV to overwrite weekly ranking of features by importance.

Of course there is a lot of room for improvement, but I'm glad the importance of features for the model looks coherent at first sight. Top 15 features, the ones selected by the model:

- erte = Record of Temporary Employment Regulation
- emergencia sanitaria = healthcare emegercy
- cáritas = a crhistian ONG helping the most vulnerable
- crecimiento económico = economical growth 
- videocall
- remote working
- Skype

In [ ]:
# Ranking of how important are the following keywords to infer in Google searches in Spain
# the keyword "unemployment"
features=pd.DataFrame()
features["features"]=X.columns
features["top_important"]=rfecv.ranking_
features.sort_values(by=["top_important"], inplace=True)
features.reset_index(drop=True, inplace=True)
features.to_csv("input/ranking_of_features.csv")
features.head(20)
# with a linear regression the number of important features was reduced. It also had better performance.